In [1]:
import sys
import os
os.chdir("..")

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

In [2]:
import pandas as pd
from src.modeling import preprocess_data
from src.modeling import train_models
from src.evaluation import evaluate_models

# Load data
X_train, X_test, y_train, y_test = preprocess_data.load_modeling_data()

In [ ]:
# Train models
rf_model = train_models.train_random_forest(X_train, y_train)
xgb_model = train_models.train_xgboost(X_train, y_train)
lgb_model = train_models.train_lightgbm(X_train, y_train)

# For MLP, we need to scale X_train first
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp_model = train_models.train_mlp(pd.DataFrame(X_train_scaled), y_train)

# Compare models
models = {
    "Random Forest": rf_model,
    "XGBoost": xgb_model,
    "LightGBM": lgb_model,
    "MLP": mlp_model
}

# Evaluate
results_df = compare_models(models, X_test, y_test)
display(results_df)